In [ ]:
#pip install requests

In [ ]:
#pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

driver_path = "chromedriver-win64/chromedriver" #크롬 드라이버 사용
service = Service(excutable_path = driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

#review_url이라고 써있는 메모장에 있는 url을 하나씩 집어넣었습니다.
url = "https://m.brand.naver.com/stanley_korea/products/10118870118?nl-query=%ED%85%80%EB%B8%94%EB%9F%AC&nl-au=a97a8ea30d5f452f83bb1e17f30b0d65&NaPm=ci%3Da97a8ea30d5f452f83bb1e17f30b0d65%7Cct%3Dma7oljq7%7Ctr%3Dnslctg%7Csn%3D10106606%7Chk%3D445e72724e62a4eaedec4dfcc8000e8ce3f02f9d"  # 크롤링할 URL로 변경
driver.get(url)
time.sleep(3)
body = driver.find_element(By.TAG_NAME, "body")

x = 20
y = 7
# Page Down 키를 x번 누름
for _ in range(x):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)  # 각 키 입력 후 대기 시간

time.sleep(2)
reviews = []
rating = []
pagenum = 8
#킨토 빈플러스 코누는 지금 있는 코드로 그대로 사용하면 되고, 스탠리 락앤락 모슈는 range를 3에서 12로 바꾸고 while true 주석을 없앤 후 단락에 있는 부분의 주석을 풀어서 실행해야 됩니다.
#킨토 빈플러스 코누는 리뷰 개수를 20으로 나눠서 range의 마지막에 넣은거라서 개수와 조금 다를 수도 있습니다.
try:
#    while True:  # 계속해서 페이지를 반복
        for page in range(1, 126):  # a[3]부터 a[11]까지 순차적으로 클릭  
            try:
                
                # 해당 페이지에서 리뷰들 가져오기
                review_elements = driver.find_elements(By.CSS_SELECTOR, "span._2L3vDiadT9") #리뷰의 css 부분분
                for element in review_elements:
                    reviews.append(element.text)
                
                for i in range(1,21): #한 페이지에 리뷰가 20개 있으므로 20개의 별점을 가져옴 
                    rating_elements = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/span/span')
                    style_value = rating_elements.get_attribute("style")
                    width_value = int(style_value.split(":")[1].strip().replace('%', '').replace(';', '').strip())
                    rating.append(width_value/20)

                # 페이지 번호 클릭 (a[3]부터 a[11]까지)
                #-----------------------------------
                # page_button = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child(" + str(page) + ")")
                # page_button.click()
                # time.sleep(2)  # 페이지 로딩 대기
                # for _ in range(6):
                #     body.send_keys(Keys.PAGE_DOWN)
                #     time.sleep(0.2)
                # time.sleep(2)
                #-----------------------------------
                next_button = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a['+ str(pagenum)+']')
                next_button.click()
                for _ in range(y):
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(0.2)
                time.sleep(2)  # '다음' 페이지로 이동 시간 기다리기
                if(pagenum<12) :
                    pagenum = pagenum+1;

            except Exception as e:
                print(f"페이지 {page} 버튼을 클릭할 수 없습니다. 더 이상 버튼이 없습니다.")
                break  # 버튼이 없으면 종료

        # '다음' 버튼 클릭 (a[12])
        #-----------------------------------
        # try:
        #     next_button = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[12]')
        #     next_button.click()
        #     for _ in range(y):
        #         body.send_keys(Keys.PAGE_DOWN)
        #         time.sleep(0.2)
        #     time.sleep(3)  # '다음' 페이지로 이동 시간 기다리기
        # except Exception:
        #     print("다음 버튼을 찾을 수 없습니다. 더 이상 페이지가 없습니다.")
        #     break  # '다음' 버튼이 없으면 종료
        #-----------------------------------

except Exception as e:
    print("오류 발생:", e)

finally:
    driver.quit()

In [ ]:
#print(reviews)

In [9]:
import pandas as pd

#reviews의의 날짜와 리뷰를 분리
data = {
    "date" : reviews[0::2],
    "review" : reviews[1::2],
    "rating" : rating
}

df = pd.DataFrame(data)
df.to_csv('konu_reviews.csv',encoding="utf-8-sig")
#url딴 브랜드 명으로 바꾼 후 csv파일로 내보내기보내기